In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Imports

In [3]:
df_air_quality = pd.read_parquet("../../data/processed_for_web/air/all_data_air_quality_from_2018.parquet")
df_air_quality_locations = pd.read_excel("../../data/processed/info/calidad_del_aire/informacion_estaciones_red_calidad_aire.xls")

FileNotFoundError: [Errno 2] No such file or directory: '../../data/processed_for_web/air/all_data_air_quality_from_2018.parquet'

## Deleting unnecessary data

In [4]:
df_air_quality_locations = df_air_quality_locations.drop(columns = ['SO2','CO','PM10','PM2_5','O3','BTX','Fecha alta','VIA_CLASE','VIA_PAR','VIA_NOMBRE','LONGITUD_ETRS89','LATITUD_ETRS89','COORDENADA_X_ETRS89','COORDENADA_Y_ETRS89','NO2'])

In [5]:
df_air_quality = df_air_quality[df_air_quality['MAGNITUD'] == 8]
df_air_quality['CODIGO'] = df_air_quality['PUNTO_MUESTREO'].str.split('_').str[0]
df_air_quality['CODIGO'] = df_air_quality['CODIGO'].astype(int)

In [6]:
df_air_quality_locations = df_air_quality_locations.rename(columns={'ESTACION':'LOCALIZACION'})

## Preprocessing

Este código realiza los siguientes pasos:

- Iteración sobre cada fila: Se procesan todas las filas del DataFrame original.
- Extracción de datos por hora: Para cada fila, se crea una nueva fila para cada hora (de 1 a 24), asignando los valores correspondientes de las columnas Hxx y Vxx.
- Construcción del nuevo DataFrame: Se utiliza una lista de diccionarios para crear un DataFrame transformado.
- Ordenación y limpieza: Se ordena el DataFrame por columnas clave y se reinician los índices.

In [7]:
# Supongamos que tu DataFrame original se llama `df`
# Creamos una lista para almacenar las filas transformadas
rows = []

# Iteramos sobre cada fila del DataFrame original
for _, row in df_air_quality.iterrows():
    for hour in range(1, 25):  # De la hora 1 a la 24
        # Construimos un diccionario para la nueva fila
        new_row = {
            'CODIGO': row['CODIGO'],
            'PROVINCIA': row['PROVINCIA'],
            'MUNICIPIO': row['MUNICIPIO'],
            'ESTACION': row['ESTACION'],
            'MAGNITUD': row['MAGNITUD'],
            'PUNTO_MUESTREO': row['PUNTO_MUESTREO'],
            'ANO': row['ANO'],
            'MES': row['MES'],
            'DIA': row['DIA'],
            'HORA': hour,  # Agregamos la hora
            'VALOR': row[f'H{hour:02}'],  # Obtenemos el valor de la columna Hxx
            'VALIDACION': row[f'V{hour:02}'],  # Obtenemos el valor de la columna Vxx
        }
        rows.append(new_row)  # Añadimos la nueva fila a la lista

# Creamos un nuevo DataFrame a partir de las filas transformadas
df_transformado = pd.DataFrame(rows)

# Creamos una nueva columna de fecha
df_transformado['FECHA'] = pd.to_datetime(df_transformado[['ANO', 'MES', 'DIA', 'HORA']].rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day', 'HORA': 'hour'}))
df_transformado = df_transformado.rename(columns = {'VALOR': 'NO2_VALUE', 'ANO': 'YEAR', 'MES': 'MONTH', 'DIA': 'DAY', 'HORA': 'HOUR'})
# Opcional: ordenamos por las columnas clave para facilitar el análisis
df_transformado = df_transformado.sort_values(by=['CODIGO','PROVINCIA', 'MUNICIPIO', 'ESTACION', 'YEAR', 'MONTH', 'DAY', 'HOUR'])

# Reiniciamos los índices para el nuevo DataFrame
df_transformado.reset_index(drop=True, inplace=True)

In [14]:
df_transformado = df_transformado[df_transformado['VALIDACION'] == 'V']

In [15]:
df_air_quality_and_locations = pd.merge(df_transformado, df_air_quality_locations, left_on='CODIGO', right_on='CODIGO')

In [16]:
df_air_quality_and_locations.to_parquet("../../data/processed/air/df_air_quality_and_locations_from_2013.parquet")